In [134]:
with open("1661-0.txt", "r", encoding='utf-8') as file:
    text = file.read()
    


In [135]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [136]:
tokenizer = Tokenizer()

In [137]:
tokenizer.fit_on_texts([text])

In [138]:
len(tokenizer.word_index)

8931

In [139]:
input_sequences = []
for sentence in text.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])
    

In [140]:
input_sequences

[[145, 4790],
 [145, 4790, 1],
 [145, 4790, 1, 1020],
 [145, 4790, 1, 1020, 4],
 [145, 4790, 1, 1020, 4, 128],
 [145, 4790, 1, 1020, 4, 128, 34],
 [145, 4790, 1, 1020, 4, 128, 34, 45],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235, 2236],
 [30, 1021],
 [30, 1021, 15],
 [30, 1021, 15, 23],
 [30, 1021, 15, 23, 1],
 [30, 1021, 15, 23, 1, 275],
 [30, 1021, 15, 23, 1, 275, 4],
 [30, 1021, 15, 23, 1, 275, 4, 394],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2, 18],
 [572, 51],
 [572, 51, 3398],
 [572, 51, 3398, 3399],
 [572, 51, 3398, 3399, 13],
 [572, 51, 3398, 3399, 13, 75],
 [572, 51, 3398, 3399, 13, 75, 817],
 [572, 51, 3398, 33

In [141]:
max_len = max([len(x) for x in input_sequences])

In [142]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [143]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       [   0,    0,    0, ..., 4790,    1, 1020],
       ...,
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358],
       [   0,    0,    0, ...,   83,  358, 1673]], dtype=int32)

In [144]:
X = padded_input_sequences[:,:-1]

In [145]:
y = padded_input_sequences[:,-1]

In [146]:
X.shape

(101619, 19)

In [147]:
y.shape

(101619,)

In [149]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=8932)

In [150]:
y.shape

(101619, 8932)

In [151]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [152]:
model = Sequential()
model.add(Embedding(8932, 100,input_length=max_len-1))
model.add(LSTM(150))
model.add(Dense(8932, activation='softmax'))

e:\Portfolio work\NextWord Predictor using LSTM\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [153]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [154]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [155]:
print("X shape:", X.shape)
print("y shape:", y.shape)

if X.shape[0] == 0 or y.shape[0] == 0:
	raise ValueError("X or y is empty. Please check your data preprocessing steps.")

history = model.fit(X, y, epochs=10, verbose=1)

X shape: (101619, 19)
y shape: (101619, 8932)
Epoch 1/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 99s 31ms/step - accuracy: 0.0611 - loss: 6.5749
Epoch 2/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 117s 37ms/step - accuracy: 0.1177 - loss: 5.5717
Epoch 3/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 113s 35ms/step - accuracy: 0.1538 - loss: 5.0948
Epoch 4/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 100s 31ms/step - accuracy: 0.1725 - loss: 4.7244
Epoch 5/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 91s 29ms/step - accuracy: 0.1955 - loss: 4.3921
Epoch 6/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 91s 29ms/step - accuracy: 0.2174 - loss: 4.0927
Epoch 7/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 92s 29ms/step - accuracy: 0.2487 - loss: 3.7806
Epoch 8/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 93s 29ms/step - accuracy: 0.2802 - loss: 3.5326
Epoch 9/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 93s 29ms/step - accuracy: 0.3204 - loss: 3.2688
Epoch 10/10
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 93s 29ms/step - accuracy: 0.3570 - loss: 3.0339


In [156]:
import numpy as np

In [157]:
def predict_next_word(seed_text, num_words=1):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word = tokenizer.index_word[np.argmax(predicted)]
        seed_text += ' ' + predicted_word
    return seed_text

In [158]:
print(predict_next_word("sherlock holmes was", 3))

sherlock holmes was pacing in his
